In [ ]:
import os
from pydub import AudioSegment
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor

# Function to sort files based on custom criteria
def sort_files(files):
    def extract_key(file_name):
        parts = file_name.split(' - ')
        if len(parts) == 2:
            base, suffix = parts[0], parts[1]
            try:
                number, index = suffix.split('_')
                number = int(number)
                index = int(index.split('.')[0])
                return base, number, index
            except ValueError:
                pass
        return file_name, 0, 0

    return sorted(files, key=extract_key)

# Function to load an audio file and speed it up by 2x
def load_and_speedup_audio(file_path):
    audio = AudioSegment.from_mp3(file_path)
    return audio.speedup(playback_speed=2.0)

# Path to the folder containing the MP3 files
folder_path = './s'

# Get all MP3 files from the folder
mp3_files = [f for f in os.listdir(folder_path) if f.endswith('.mp3')]

# Sort files based on custom criteria
sorted_files = sort_files(mp3_files)

print(sorted_files)



In [ ]:

# Function to load an audio file
def load_audio(file_path):
    return AudioSegment.from_mp3(file_path)

# Function to process and concatenate audio files in chunks
def process_in_chunks(files, chunk_size=10):
    combined = AudioSegment.empty()
    for i in tqdm(range(0, len(files), chunk_size), desc="Processing chunks", unit="chunk"):
        chunk_files = files[i:i + chunk_size]
        with ThreadPoolExecutor() as executor:
            chunk_audios = list(executor.map(load_and_speedup_audio, chunk_files))
        for audio in chunk_audios:
            combined += audio
    return combined

# Create a list of file paths
file_paths = [os.path.join(folder_path, mp3_file) for mp3_file in sorted_files]


In [3]:

# Process files in chunks
combined_audio = process_in_chunks(file_paths, chunk_size=10)


In [ ]:

# Export the combined audio to a new MP3 file
output_path = folder_path + '.mp3' 
combined_audio.export(output_path, format='mp3')

print(f"Combined MP3 file created at: {output_path}")


In [ ]:
import os
from pydub import AudioSegment
from tqdm import tqdm
from concurrent.futures import ProcessPoolExecutor

# Function to sort files based on custom criteria
def sort_files(files):
    def extract_key(file_name):
        parts = file_name.split(' - ')
        if len(parts) == 2:
            base, suffix = parts[0], parts[1]
            try:
                number, index = suffix.split('_')
                number = int(number)
                index = int(index.split('.')[0])
                return base, number, index
            except ValueError:
                pass
        return file_name, 0, 0

    return sorted(files, key=extract_key)

# Path to the folder containing the MP3 files
folder_path = './s'

# Get all MP3 files from the folder
mp3_files = [f for f in os.listdir(folder_path) if f.endswith('.mp3')]

# Sort files based on custom criteria
sorted_files = sort_files(mp3_files)

print(sorted_files)

# Function to load an audio file and speed it up by 2x
def load_and_speedup_audio(file_path):
    audio = AudioSegment.from_mp3(file_path)
    return audio.speedup(playback_speed=2.0)

# Function to process and concatenate audio files in chunks
def process_in_chunks(files, chunk_size=10):
    combined = AudioSegment.empty()
    with ProcessPoolExecutor() as executor:
        for i in tqdm(range(0, len(files), chunk_size), desc="Processing chunks", unit="chunk"):
            chunk_files = files[i:i + chunk_size]
            chunk_audios = list(executor.map(load_and_speedup_audio, chunk_files))
            for audio in chunk_audios:
                combined += audio
    return combined

# Function to export combined audio in chunks to avoid file size limits
def export_in_chunks(audio, output_path, chunk_duration=5*60*60*1000): # 5 hours per chunk
    chunk_count = len(audio) // chunk_duration + 1
    for i in range(chunk_count):
        start_time = i * chunk_duration
        end_time = start_time + chunk_duration
        chunk = audio[start_time:end_time]
        chunk.export(f"{output_path}_part{i+1}.mp3", format="mp3")


In [ ]:

# Create a list of file paths
file_paths = [os.path.join(folder_path, mp3_file) for mp3_file in sorted_files]

# Process files in chunks
combined_audio = process_in_chunks(file_paths, chunk_size=10)


In [ ]:

# Export the combined audio in smaller chunks to avoid size limits
output_path = 'combined_audio'
export_in_chunks(combined_audio, output_path)

print(f"Combined and sped-up MP3 file created in chunks with base name: {output_path}")
